In [1]:
import pandas as pd
import numpy as np
from scipy import stats

from matplotlib import pyplot as plt
plt.style.use('ggplot')

import seaborn as sns
from scipy.stats import pearsonr

In [29]:
df = pd.read_csv('../house-prices-advanced-regression-techniques/test.csv')

# Land Variables

In [31]:
land_df = df[['LotArea', 'LotFrontage', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 
              'Neighborhood', 'Condition1', 'Condition2']]

In [32]:
#Impute LotFrontage missing values

land_df['LotFrontage'] = land_df['LotFrontage'].fillna(0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
shape = pd.get_dummies(land_df['LotShape'], prefix='LotShape', prefix_sep='_')
shape = shape.drop('LotShape_IR3', axis=1)

contour = pd.get_dummies(land_df['LandContour'], prefix='LandContour', prefix_sep='_')
contour = contour.drop('LandContour_Bnk', axis=1)

config = pd.get_dummies(land_df['LotConfig'], prefix='LotConfig', prefix_sep='_')
config = config.drop('LotConfig_FR3', axis=1)

slope = pd.get_dummies(land_df['LandSlope'],  prefix='LandSlope', prefix_sep='_')
slope = slope.drop('LandSlope_Mod', axis=1)

neigh = pd.get_dummies(land_df['Neighborhood'], prefix='Neighborhood', prefix_sep='_')
neigh = neigh.drop('Neighborhood_SWISU', axis=1)

cond1 = pd.get_dummies(land_df['Condition1'], prefix='Condition1', prefix_sep='_')
cond1 = cond1.drop('Condition1_Artery', axis=1)

cond2 = pd.get_dummies(land_df['Condition2'], prefix='Condition2', prefix_sep='_')
cond2 = cond2.drop('Condition2_Artery', axis=1)


In [36]:
land_df_cont = land_df[['LotArea', 'LotFrontage']]

land_df_dum = pd.concat([land_df_cont, shape, contour, config, slope, neigh, cond1, cond2], axis=1)



In [38]:
land_final = pd.concat([shape, config, neigh, cond1, land_df_cont], axis=1)
land_final

,LotShape_IR1,LotShape_IR2,LotShape_Reg,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,...,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,LotArea,LotFrontage
0,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,11622,80.0
1,1,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,14267,81.0
2,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,13830,74.0
3,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,9978,78.0
4,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,5005,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1936,21.0
1455,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1894,21.0
1456,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,20000,160.0
1457,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,10441,62.0


# Exterior Variables

In [54]:
ext_df = df[['PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence']]

In [55]:
ext_df

,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence
0,Y,140,0,0,0,120,0,NaN,MnPrv
1,Y,393,36,0,0,0,0,NaN,NaN
2,Y,212,34,0,0,0,0,NaN,MnPrv
3,Y,360,36,0,0,0,0,NaN,NaN
4,Y,0,82,0,0,144,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1454,Y,0,0,0,0,0,0,NaN,NaN
1455,Y,0,24,0,0,0,0,NaN,NaN
1456,Y,474,0,0,0,0,0,NaN,NaN
1457,Y,80,32,0,0,0,0,NaN,MnPrv


In [56]:
ext_df['PavedDrive'] = ext_df['PavedDrive'].replace('Y', 1)
ext_df['PavedDrive'] = ext_df['PavedDrive'].replace('N', 0)
ext_df['PavedDrive'] = ext_df['PavedDrive'].replace('P', 1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [57]:
ext_df['Fence'] = ext_df['Fence'].replace('MnPrv', 1)
ext_df['Fence'] = ext_df['Fence'].replace('GdPrv', 1)
ext_df['Fence'] = ext_df['Fence'].replace('GdWo', 1)
ext_df['Fence'] = ext_df['Fence'].replace('MnWw', 1)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [58]:
# Change NA's in whole dataframe to 0s
ext_df = ext_df.fillna(0)
ext_df

,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence
0,1,140,0,0,0,120,0,0,1.0
1,1,393,36,0,0,0,0,0,0.0
2,1,212,34,0,0,0,0,0,1.0
3,1,360,36,0,0,0,0,0,0.0
4,1,0,82,0,0,144,0,0,0.0
...,...,...,...,...,...,...,...,...,...
1454,1,0,0,0,0,0,0,0,0.0
1455,1,0,24,0,0,0,0,0,0.0
1456,1,474,0,0,0,0,0,0,0.0
1457,1,80,32,0,0,0,0,0,1.0


In [59]:
# impute 1 for PoolArea
df_hasPool = ext_df[ext_df['PoolArea']>0]
df_hasPool['PoolArea'] = 1
df_NoPool = ext_df[ext_df['PoolArea']==0]

ext_df = pd.concat([df_hasPool, df_NoPool])
ext_df.drop('PoolQC', axis=1, inplace=True)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
ext_df

,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,Fence
514,1,154,65,0,0,216,1,0.0
960,1,225,0,0,0,0,1,1.0
1043,1,0,195,1012,0,0,1,0.0
1113,1,216,64,169,0,0,1,0.0
1139,0,483,266,0,0,0,1,1.0
...,...,...,...,...,...,...,...,...
1454,1,0,0,0,0,0,0,0.0
1455,1,0,24,0,0,0,0,0.0
1456,1,474,0,0,0,0,0,0.0
1457,1,80,32,0,0,0,0,1.0


In [61]:
# Final cleaning of ext_df
ext_final = ext_df.rename(columns={"PoolArea": "Pool"})


In [62]:
# FINAL Dataframe, land and exterior features combined
land_ext_final = pd.concat([land_final, ext_final], axis=1)
land_ext_final

,LotShape_IR1,LotShape_IR2,LotShape_Reg,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,...,LotArea,LotFrontage,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Pool,Fence
0,0,0,1,0,0,0,1,0,0,0,...,11622,80.0,1,140,0,0,0,120,0,1.0
1,1,0,0,1,0,0,0,0,0,0,...,14267,81.0,1,393,36,0,0,0,0,0.0
2,1,0,0,0,0,0,1,0,0,0,...,13830,74.0,1,212,34,0,0,0,0,1.0
3,1,0,0,0,0,0,1,0,0,0,...,9978,78.0,1,360,36,0,0,0,0,0.0
4,1,0,0,0,0,0,1,0,0,0,...,5005,43.0,1,0,82,0,0,144,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0,0,1,0,0,0,1,0,0,0,...,1936,21.0,1,0,0,0,0,0,0,0.0
1455,0,0,1,0,0,0,1,0,0,0,...,1894,21.0,1,0,24,0,0,0,0,0.0
1456,0,0,1,0,0,0,1,0,0,0,...,20000,160.0,1,474,0,0,0,0,0,0.0
1457,0,0,1,0,0,0,1,0,0,0,...,10441,62.0,1,80,32,0,0,0,0,1.0


In [64]:
land_ext_final.to_csv('land_ext_test.csv')